#  ==================================
# IMPORTS
#  ==================================

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

sns.set_theme()

# jupyter notebook full-width display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# no text wrapping
display(HTML("<style>.dataframe td { white-space: nowrap; }</style>"))

# pandas formatting
pd.set_option('display.float_format', '{:.3f}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', 400)

In [2]:
# import dataframes
df_SD = pd.read_pickle('df_SD.pickle')
df_FD = pd.read_pickle('df_FD.pickle')
df_LF = pd.read_pickle('df_LF.pickle')
df_Site = pd.read_pickle('df_Site.pickle')
df_TrapSupervisors = pd.read_pickle('df_TrapSupervisors.pickle')

### Helper Function to UPDATE ID to Match Updated DATETIME and SITE

In [247]:
def create_new_id(row):
    """only run before disambiguating month portion of ids for FD and LF"""
    return str(row.id)[0:4] + str(row.DATETIME.month).rjust(2,'0') + str(row.DATETIME.day).rjust(2, '0') + str(row.SITE1).rjust(2, '0')

#  ==================================
# FISH DETAILS
#  ==================================

## Combine Age 2 and Age 3 Columns

In [3]:
# check if age2 and age3 occur at the same time
(
    df_FD[df_FD['Ager_2'].notnull() & df_FD['Ager_3'].notnull()].shape[0],
    df_FD[df_FD['AGE_2'].notnull() & df_FD['AGE_3'].notnull()].shape[0],
    df_FD[df_FD['FSP_2'].notnull() & df_FD['FSP_3'].notnull()].shape[0], 
    df_FD[df_FD['Comments_2'].notnull() & df_FD['Comments_3'].notnull()].shape[0]
)

(0, 0, 0, 0)

In [4]:
# how many Ager 2 columns
df_FD[df_FD.Ager_2.notnull() | df_FD.AGE_2.notnull() | df_FD.FSP_2.notnull() | df_FD.Comments_2.notnull()].shape[0]

1683

In [5]:
# how many Ager 3 columns
df_FD[df_FD.Ager_3.notnull() | df_FD.AGE_3.notnull() | df_FD.FSP_3.notnull() | df_FD.Comments_3.notnull()].shape[0]

2339

In [6]:
# combine aging 2 and aging 3 into one set of data
df_FD['Ager_2'] = df_FD['Ager_2'].fillna(df_FD['Ager_3'])
df_FD['AGE_2'] = df_FD['AGE_2'].fillna(df_FD['AGE_3'])
df_FD['FSP_2'] = df_FD['FSP_2'].fillna(df_FD['FSP_3'])
df_FD['Comments_2'] = df_FD['Comments_2'].fillna(df_FD['Comments_3'])

In [7]:
# how many Ager 2 columns
df_FD[df_FD.Ager_2.notnull() | df_FD.AGE_2.notnull() | df_FD.FSP_2.notnull() | df_FD.Comments_2.notnull()].shape[0]

4022

## FLAG_SITE: ambiguous site data

In [8]:
# TODO: if site is changed away from SITE1, update id to match

# df_FD.loc[df_FD.FLAG_SITE==True]

## FLAG_AM_PM_PERIOD: leave as is

## FLAG_SEX: B and A? -> null

In [9]:
df_FD.loc[df_FD.FLAG_SEX==True, 'SEX'] = np.nan

In [10]:
df_FD.loc[:, 'SEX'].unique()

array(['F', 'M', 'U', nan], dtype=object)

## FLAG_MATURITY: 44=4, 0=null

In [11]:
df_FD.loc[df_FD.MATURITY==44, 'MATURITY'] = 4
df_FD.loc[df_FD.MATURITY==0, 'MATURITY'] = np.nan

## FLAG_FSP_1: 33=3

In [12]:
df_FD.loc[df_FD.FSP_1==33, 'FSP_1'] = 3

## FLAG_FL_STD: 10x off

In [13]:
# CONFIRM
df_FD.loc[df_FD.FLAG_FL_STD==True, 'FL_STD'] *= 10
df_FD.loc[df_FD.FLAG_FL_STD==True, 'FL_STD']

index
34891   261.000
34892   253.000
34893   248.000
34894   258.000
34895   261.000
          ...  
35244   240.000
35245   250.000
35246   200.000
35247   204.000
35248   241.000
Name: FL_STD, Length: 264, dtype: float64

## FLAG_FL_WET_FROZEN: 5 typos

In [14]:
# CONFIRM
df_FD.loc[df_FD.FLAG_FL_WET_FROZEN==True, ['FL_WET', 'FL_FROZEN']] = np.nan
df_FD.loc[df_FD.FLAG_FL_WET_FROZEN==True, ['FL_WET', 'FL_FROZEN']]

,FL_WET,FL_FROZEN
index,,
35048,<NA>,<NA>
35161,<NA>,<NA>
35241,<NA>,<NA>
35246,<NA>,<NA>
35247,<NA>,<NA>


## FLAG_WEIGHT_OUTLIER: 

In [15]:
df_FD.loc[df_FD.FLAG_WEIGHT_OUTLIER==True, 'WEIGHT']

index
718      519.000
914      539.000
1348     501.000
1800     512.000
5324     503.000
34988   1934.000
Name: WEIGHT, dtype: float64

In [16]:
# CONFIRM
df_FD.loc[df_FD.WEIGHT>1000, 'WEIGHT'] = np.nan
df_FD.loc[df_FD.WEIGHT>1000, 'WEIGHT']

Series([], Name: WEIGHT, dtype: float64)

## FLAG_GONAD_OUTLIER:

In [17]:
# CONFIRM
df_FD.loc[df_FD.FLAG_GONAD_OUTLIER==True, 'GONAD_WEIGHT'] = np.nan
df_FD.loc[df_FD.FLAG_GONAD_OUTLIER==True, 'GONAD_WEIGHT']

index
5590    NaN
14050   NaN
23051   NaN
23498   NaN
25251   NaN
26652   NaN
32829   NaN
Name: GONAD_WEIGHT, dtype: float64

## FLAG_MULTIPLE_SAMPLE_POSSIBILITIES and FLAG_MISNUMBERED_FISH_DETAILS: 

In [18]:
# df_FD.loc[df_FD.FLAG_MULTIPLE_SAMPLE_POSSIBILITIES==True]

In [105]:
# df_FD.loc[df_FD.FLAG_MISNUMBERED_FISH_DETAILS==True]

## FLAG_LEN_WT_RATIO_OUTLIER: 

In [19]:
# df_FD.loc[df_FD.FLAG_LEN_WT_RATIO_OUTLIER==True, ['FL_STD', 'WEIGHT']]

#  ==================================
# LENGTH FREQUENCIES
#  ==================================

## FLAG_SITE: ambiguous site

In [116]:
# TODO: if site is changed away from SITE1, update id to match

# df_LF.loc[df_LF.FLAG_SITE==True]

## FLAG_AM_PM_PERIOD: leave as is

#  ==================================
# SAMPLES
#  ==================================

## FLAG_DATETIME: null datetime

In [117]:
# TODO: if date is changed, update id to match

# df_SD.loc[df_SD.FLAG_DATETIME==True]

## FLAG_HOURS_FISHED: hours_fished = "maximum "

In [22]:
# CONFIRM
df_SD.loc[df_SD.FLAG_HOURS_FISHED==True, 'hours_fished'] = 18
df_SD.loc[df_SD.FLAG_HOURS_FISHED==True, 'hours_fished']

13695    18
13705    18
13715    18
13725    18
13736    18
13740    18
13748    18
13757    18
13777    18
13780    18
13793    18
13810    18
13823    18
13827    18
13840    18
13856    18
13874    18
13886    18
13893    18
13908    18
13921    18
13925    18
13944    18
13958    18
13961    18
13973    18
13992    18
14002    18
Name: hours_fished, dtype: object

## FLAG_SITE: ambiguous site data

In [23]:
# TODO: if site is updated, update id to match

# these are currently importing as 1A
df_SD.loc[df_SD.FLAG_SITE==True, 'SITE_NO']

5982    1A,8
5999    1A,8
6006    1A,8
6025    1A,8
6036    1A,8
6049    1A,8
6064    1A,8
6092    1A,8
6106    1A,8
6112    1A,8
6118    1A,8
6125    1A,8
6130    1A,8
6138    1A,8
6153    1A,8
6162    1A,8
6193    1A,8
6198    1A,8
6208    1A,8
6215    1A,8
6230    1A,8
6251    1A,8
6256    1A,8
Name: SITE_NO, dtype: object

## FLAG_AM_PM_PERIOD_DISCREPANCIES: FD and LF inconsistent
#### SAMPLES, LENGTH FREQUENCIES, and FISH DETAILS all flagged
* The samples will be where the AM PM data will be stored, so they are the only updates needed, outside of ghost samples, which can be updated as information becomes available.

In [24]:
df_SD.loc[df_SD.FLAG_AM_PM_PERIOD_DISCREPANCIES==True]

,DIST,RIVER,NAME,code,GEAR,SITE_NO,no_nets,YEAR,MM,DD,Week,catch_lbs,catch_kg,hours_fished,zone,last_name,comments,bycatch_sbass,bycatch_shad,bycatch_other,DATETIME,SITE1,SITE2,remarks,id,total_fish_preserved,total_fish_measured,AM_PM_PERIOD,wt_lbs,FLAG_DATETIME,FLAG_HOURS_FISHED,FLAG_SITE,FLAG_AM_PM_PERIOD_DISCREPANCIES
5130,2,SWMARG,Daniel Stewart,NaN,81,33,NaN,1993,5,13,3,1100.000,499.000,NaN,upper,STEWART,NaN,NaN,NaN,NaN,1993-05-13,33,<NA>,NaN,1993051333,34.000,210.000,AM,115.000,NaN,NaN,NaN,True
8200,2,SWMARG,Pierre Chiasson,NaN,81,26,1,2001,6,15,7,500.000,226.800,NaN,lower,Chiasson,NaN,NaN,NaN,NaN,2001-06-15,26,<NA>,NaN,2001061526,51.000,283.000,AM,136.000,NaN,NaN,NaN,True
10211,2,SWMARG,Pierre Chiasson,NaN,81,26,1,2007,6,2,5,9600.000,4354.500,8,lower,Chiasson,NaN,NaN,NaN,NaN,2007-06-02,26,<NA>,NaN,2007060226,32.000,290.000,AM,130.000,NaN,NaN,NaN,True
13373,2,SWMARG,Donelda M Gillis,NaN,81,47,1,2014,6,10,7,450.000,204.100,7,upper,Gillis,NaN,NaN,NaN,NaN,2014-06-10,47,<NA>,NaN,2014061047,38.000,246.000,AM,89.000,NaN,NaN,NaN,True


In [25]:
# df_FD.loc[df_FD.FLAG_AM_PM_PERIOD_DISCREPANCIES==True]

In [26]:
# df_LF.loc[df_LF.FLAG_AM_PM_PERIOD_DISCREPANCIES==True]

## FLAG_NO_MATCHING_SAMPLE: no SAMPLE matching LF and/or FD
* these will end up being matched with ghost samples (below)

In [27]:
# df_FD.loc[df_FD.FLAG_NO_MATCHING_SAMPLE==True]

In [28]:
# df_LF.loc[df_LF.FLAG_NO_MATCHING_SAMPLE==True]

## MAKE GHOST SAMPLES
### for unmatched fish details and length frequencies
### NOTE:
Some ambiguous length frequencies and fish details match with eachother. However, we do not know which sample and length frequency are from the same sample. Therefore, we should make sure none of the ambiguous entries ever match automatically, and they are matched manually (eventually) in dm_apps, once that feature is implemented.

Although there are no ambiguous samples that match with length frequencies or fish details, just for throroughness, we should revise id to avoid this possibility as well. 

Therefore, we will arbitrarily add 20 to months in ids of ambiguous fish details, and 40 to months in ids of ambiguous length frequencies. These numbers have no possibility to match a false positive, and they still uniquely identify their date and site.

In [29]:
# to disambiguate ids, add 20 to all months for FD with id>2024000000, add 40 to all months for LF with id>2024000000

# CONFIRM (period is not highly consistent, probably match, but left as False for now)
if match_FD_LF_confirmed_using_period := False:  
    # these are the only ambiguous fish details and length frequencies that appear to match after considering AM/PM period
    matched_period = [3010051926, 4010051926, 3010052726, 4010052726]  # matched using period
    df_FD.loc[(df_FD.id>2024000000) & (~df_FD.id.isin(matched_period)), 'id'] = df_FD.loc[df_FD.id>2024000000, 'id'] + 200000
    df_LF.loc[(df_LF.id>2024000000) & (~df_LF.id.isin(matched_period)), 'id'] = df_LF.loc[df_LF.id>2024000000, 'id'] + 400000
else:
    df_FD.loc[df_FD.id>2024000000, 'id'] = df_FD.loc[df_FD.id>2024000000, 'id'] + 200000
    df_LF.loc[df_LF.id>2024000000, 'id'] = df_LF.loc[df_LF.id>2024000000, 'id'] + 400000

# verified samples are all sample ids before creating ghost samples
verified_samples = set(df_SD.id)

# add null samples where no match exists - Fish Details
df_SD = pd.concat([
    df_SD, 
    df_FD[~df_FD.id.isin(df_SD.id)].drop_duplicates('id')[['id', 'DATETIME', 'SITE1']]
]).reset_index(drop=True)

# add null samples where no match exists - Length Frequencies
df_SD = pd.concat([
    df_SD, 
    df_LF[~df_LF.id.isin(df_SD.id)].drop_duplicates('id')[['id', 'DATETIME', 'SITE1']]
]).reset_index(drop=True)

# add a Flag to indicate ghost samples for import
df_SD.loc[~df_SD.id.isin(verified_samples), 'FLAG_GHOST_SAMPLE'] = True

# add remarks
df_SD.loc[~df_SD.id.isin(verified_samples), 'remarks'] = 'GHOST SAMPLE, created to match with unmatched Fish Details and/or Length Frequencies'

## REMERGE JOINED DATA
(previously merged without ghost samples, remerge with ghost data)

In [30]:
# drop previously merged columns (to be re-merged)
df_SD = df_SD.drop(['total_fish_preserved', 'total_fish_measured', 'AM_PM_PERIOD', 'wt_lbs'], axis=1)

# JOIN with Fish Details table to get total_fish_preserved
# NOTE: this is an estimate, assuming all fish details are accounted for. This is the best information available
df_SD = pd.merge(
    df_SD, 
    pd.merge(df_SD, df_FD, on='id').groupby('id').count()['FISH_NO'].reset_index(),
    on='id',
    how='left'
).rename({'FISH_NO':'total_fish_preserved'}, axis=1)

# JOIN with Length Frequencies table to get total_fish_measured
# NOTE: this is an estimate, assuming all length frequencies are accounted for. This is the best information available
df_SD = pd.merge(
    df_SD, 
    pd.merge(df_SD, df_LF, on='id').groupby('id').sum()['freq'].reset_index(),
    on='id',
    how='left'
).rename({'freq':'total_fish_measured'}, axis=1)

# # JOIN with Fish Details and Length Frequencies to get AM_PM_PERIOD 
# # NOTE: discrepancies flagged between Length Frequencies and Fish Details
df_period = pd.merge(
    df_FD[df_FD.PERIOD.notnull()].groupby('id').first().reset_index()[['id', 'PERIOD']],
    df_LF[df_LF.period.notnull()].groupby('id').first().reset_index()[['id', 'period']], 
    on='id',
    how='outer'
)
df_period['AM_PM_PERIOD'] = df_period['PERIOD'].fillna(df_period['period'])

df_SD = pd.merge(
    df_SD, 
    df_period[['id', 'AM_PM_PERIOD']],
    on='id',
    how='left'
)

# JOIN with Length Frequency table to get sample weight
df_SD = pd.merge(
    df_SD, 
    df_LF[['id', 'wt_lbs']],
    on='id', 
    how='left'  # all samples 
).drop_duplicates().reset_index(drop=True)

# Fix Datetime Format

In [31]:
# some times merged into DATETIME from other df via ghost samples () 
# reduce to date and AM / PM period
df_SD['DATETIME'] = pd.to_datetime(df_SD['DATETIME'].dt.date)

#  ==================================
# LF Grouped
#  ==================================

## Recreate df_LF_grouped with new ambiguous ids for LF entries (+40 to month if ambiguous)

In [32]:
# group by length bins, include only imported columns 
df_LF_grouped = df_LF.groupby(['id', 'length_bin_id']).sum('freq').reset_index()[['freq', 'length_bin_id', 'id']].rename({
    'freq': 'count', 
    'length_bin_id': 'length_bin_id', 
    'id': 'sample_id'
}, axis=1).reset_index(drop=True)[['sample_id', 'length_bin_id', 'count']]

In [33]:
# ambiguous entries, month bit (+40 to disambiguate)
min([int(str(x)[4:6]) for x in list(df_LF_grouped[df_LF_grouped.sample_id>2024000000].sample_id.unique())])

45

In [34]:
# regular entries, month bit
max([int(str(x)[4:6]) for x in list(df_LF_grouped[df_LF_grouped.sample_id<2024000000].sample_id.unique())])

6

#  ==================================
# SAVE DATA
#  ==================================

In [35]:
# reorder columns for clarity - all flags at the end
df_SD = df_SD[[x for x in list(df_SD.columns) if 'FLAG' not in str(x)] + [x for x in list(df_SD.columns) if 'FLAG' in str(x)]]
df_FD = df_FD[[x for x in list(df_FD.columns) if 'FLAG' not in str(x)] + [x for x in list(df_FD.columns) if 'FLAG' in str(x)]]
df_LF = df_LF[[x for x in list(df_LF.columns) if 'FLAG' not in str(x)] + [x for x in list(df_LF.columns) if 'FLAG' in str(x)]]

In [36]:
# save pickle files of dataframes
if save_pickles := True:
    df_SD.to_pickle('df_SD_cleaned.pickle')
    df_FD.to_pickle('df_FD_cleaned.pickle')
    df_LF.to_pickle('df_LF_cleaned.pickle')
    df_LF_grouped.to_pickle('df_LF_grouped_cleaned.pickle')
    df_Site.to_pickle('df_Site_cleaned.pickle')
    df_TrapSupervisors.to_pickle('df_TrapSupervisors_cleaned.pickle')

# save files to this workbook drive
if save_csv := False:
    df_SD.to_csv('gaspereau_sample_data_cleaned.csv', index=False)
    df_FD.to_csv('gaspereau_fish_details_cleaned.csv', index=False)
    df_LF.to_csv('gaspereau_length_frequencies_cleaned.csv', index=False)
    df_Site.to_csv('gaspereau_sites_cleaned.csv', index=False)
    df_TrapSupervisors.to_csv('gaspereau_trap_supervisors_cleaned.csv', index=False)
    df_LF_grouped.to_csv('gaspereau_LF_grouped_cleaned.csv', index=False)
    
# upload to temp folder for import into Kevin's local dm_apps
import_file_location = 'C:\\Users\\CARRK\\Documents\\Repositories\\dm_app_root\\dm_apps\\herring\\temp\\'

if upload_csv_to_temp_folder := True:
    df_SD.to_csv(import_file_location + 'gaspereau_sample_data.csv', index=False)
    df_FD.to_csv(import_file_location + 'gaspereau_fish_details.csv', index=False)
    df_Site.to_csv(import_file_location + 'gaspereau_sites.csv', index=False)
    df_TrapSupervisors.to_csv(import_file_location + 'gaspereau_trap_supervisors.csv', index=False)
    df_LF_grouped.to_csv(import_file_location + 'gaspereau_LF_grouped.csv', index=False)

# Check Import Issues

In [37]:
checkthese = [
    1993052010, 1993053010, 2016053100, 2016061400, 2016062100, 2017053092, 2017060692, 2018052900, 2018060800, 
    2018062700, 2019060794, 2019060793, 2019061493, 2019062594, 1989060100, 1989060200, 1989060700, 2011060414, 
    2016053195, 2016061495, 2016062195, 2018052992, 2018062792, 2018060892, 2021052092, 2021060192, 2021060392, 
    2021061592, 2021062392
]

# these were rejected then recreated
len(list(df_SD[df_SD.id.isin(checkthese)].id)), len(checkthese)

(29, 29)

In [38]:
# why were they rejected
df_SD[df_SD.id.isin(checkthese)].SITE1.unique()

array([<NA>, 14, 95, 92, 93, 94, '10'], dtype=object)

In [39]:
# none of the sites match
df_Site.site.unique()

array([1, '1A', '1B', 2, 4, 5, 6, 7, 8, 9, 11, 12, 15, 17, 18, 19, 20, 21,
       23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 40,
       41, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 55, 56, 57, 58, 59,
       60, 61, 62, 63, 64, 65, 66, 67, 68], dtype=object)

In [40]:
# SOLUTION: if a site is unmatched, make it None, don't reject it
# didn't happen, but similar for trap_supervisor not matching FK table

# After Import Checks

In [41]:
rejected_FD = list(df_FD[df_FD.FLAG_MISNUMBERED_FISH_DETAILS==True].id.unique())
rejected_SD = list(df_SD[df_SD.FLAG_DATETIME==True].id.unique())
rejected_id = rejected_FD + rejected_SD

In [42]:
# -> CORRECT
# how many total samples were imported?
df_SD[df_SD.FLAG_DATETIME.isnull()].shape[0]

15485

In [43]:
# what about LF, FD, both?
(
    df_SD[(df_SD.FLAG_DATETIME.isnull()) & (df_SD.id.isin(df_LF.id.unique()))].shape[0],
    df_SD[(df_SD.FLAG_DATETIME.isnull()) & (df_SD.id.isin(df_FD.id.unique()))].shape[0],
    df_SD[(df_SD.FLAG_DATETIME.isnull()) & (df_SD.id.isin(df_LF.id.unique())) & (df_SD.id.isin(df_FD.id.unique()))].shape[0]
)
# -> 916, 1149, 803
# yes, meh, yes

(916, 1156, 803)

In [ ]:
# -> CORRECT
# why are we missing 7 FD? were they rejected?
# -> YES, exactly 7 were rejected (invalid kwargs)

In [44]:
# without any FD or LF?
df_SD[~(df_SD.id.isin(df_LF.id.unique())) & ~(df_SD.id.isin(df_FD.id.unique())) & (df_SD.FLAG_DATETIME.isnull())].shape[0]
# -> 14223
# close, but how are there more in dm_apps?

14216

In [55]:
# maybe the rejected FD were still imported
rejected_FD = [2993252737, 2993252952, 2993260933, 2998250105, 3010252925, 3010250541, 3010261141] # all of these samples were imported as ghost samples
df_SD[~(df_SD.id.isin(df_LF.id.unique())) & (~(df_SD.id.isin(df_FD.id.unique())) | df_SD.id.isin(rejected_FD)) & (df_SD.FLAG_DATETIME.isnull())].shape[0]
# -> CORRECT (yuck code, but it matches)

14223

In [45]:
# -> CORRECT
# how many ambiguous samples?  6
# do these match with FD or LF?  No. (correct)
df_SD[~(df_SD.FLAG_GHOST_SAMPLE == True) & (df_SD.id > 2024000000) & (df_SD.FLAG_DATETIME.isnull())][['DATETIME']+list(df_SD.columns)]

,DATETIME,DIST,RIVER,NAME,code,GEAR,SITE_NO,no_nets,YEAR,MM,DD,Week,catch_lbs,catch_kg,hours_fished,zone,last_name,comments,bycatch_sbass,bycatch_shad,bycatch_other,DATETIME,SITE1,SITE2,remarks,id,total_fish_preserved,total_fish_measured,AM_PM_PERIOD,wt_lbs,FLAG_DATETIME,FLAG_HOURS_FISHED,FLAG_SITE,FLAG_AM_PM_PERIOD_DISCREPANCIES,FLAG_GHOST_SAMPLE
2452,1988-05-23,2.000,SWMARG,Michael D Gillis,NaN,81.000,48,NaN,1988,5,23,<NA>,7015.000,3182.000,13,upper,Gillis,NaN,NaN,NaN,NaN,1988-05-23,48,<NA>,NaN,2988052348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2535,1988-05-23,2.000,SWMARG,Michael D Gillis,NaN,81.000,48,NaN,1988,5,23,<NA>,4008.000,1818.000,13,upper,Gillis,NaN,NaN,NaN,NaN,1988-05-23,48,<NA>,NaN,3988052348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6514,1997-06-12,2.000,SWMARG,Michael D Gillis,NaN,81.000,58,1,1997,6,12,7,900.000,408.200,NaN,upper,Gillis,NaN,NaN,NaN,NaN,1997-06-12,58,<NA>,NaN,2997061258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6515,1997-06-12,2.000,SWMARG,Michael D Gillis,NaN,81.000,58,1,1997,6,12,7,200.000,90.700,NaN,upper,Gillis,NaN,NaN,NaN,NaN,1997-06-12,58,<NA>,NaN,3997061258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9176,2004-06-10,2.000,SWMARG,Charles McDaniel,NaN,81.000,1,1,2004,6,10,7,1150.000,521.600,15,lower,McDaniel,NaN,NaN,NaN,NaN,2004-06-10,1,<NA>,NaN,3004061001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9177,2004-06-10,2.000,SWMARG,Charles McDaniel,NaN,81.000,1,1,2004,6,10,7,1125.000,510.300,6.5,lower,McDaniel,NaN,NaN,NaN,NaN,2004-06-10,1,<NA>,NaN,4004061001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
# -> CORRECT
# how many ghost sites? with LF? FD? both?
(
    df_SD[df_SD.FLAG_GHOST_SAMPLE == True].shape[0],
    df_SD[(df_SD.FLAG_GHOST_SAMPLE == True) & (df_SD.id.isin(df_LF.id.unique()))].shape[0],
    df_SD[(df_SD.FLAG_GHOST_SAMPLE == True) & (df_SD.id.isin(df_FD.id.unique())) & ~(df_SD.id.isin(rejected_id))].shape[0],
    df_SD[(df_SD.FLAG_GHOST_SAMPLE == True) & (df_SD.id.isin(df_LF.id.unique())) & (df_SD.id.isin(df_FD.id.unique())) & ~(df_SD.id.isin(rejected_id))].shape[0]
)
# -> 303, 185, 241, 130

(303, 185, 241, 130)